# Lambda Calculus
Lambda calculus (also written as λ-calculus) is a formal system in mathematical logic for expressing computation based on function abstraction and application using variable binding and substitution. It is a universal model of computation that can be used to simulate any Turing machine. (i.e. it is turing complete). It was first introduced by mathematician Alonzo Church in the 1930s as part of his research of the foundations of mathematics [(wikipedia)](https://en.wikipedia.org/wiki/Lambda_calculus).

Our goal in this notebook will be to implement an untyped λ-calculus with [Church Encodings](https://en.wikipedia.org/wiki/Church_encoding).

## Detour: Python Lambdas
In computer programming, an anonymous function (function literal, lambda abstraction, or lambda expression) is a function definition that is not bound to an identifier. Anonymous functions are often arguments being passed to higher-order functions, or used for constructing the result of a higher-order function that needs to return a function. If the function is only used once, or a limited number of times, an anonymous function may be syntactically lighter than using a named function. Anonymous functions are ubiquitous in functional programming languages and other languages with first-class functions, where they fulfill the same role for the function type as literals do for other data types [(wikipedia)](https://en.wikipedia.org/wiki/Anonymous_function).

In python, the simplest lambda can be created as follows:

In [1]:
lambda x: 0

<function __main__.<lambda>>

In this expression `lambda` declares the function. The `x` represents the input argument of the function. The expression that appears after the colon represents the output of the function. In this case 0 will be the output regardless of input. This can be seen by calling the function on different inputs.

In [2]:
print (lambda x: 0)(0)
print (lambda x: 0)(1)

0
0


Alternatively, we can bind this function to an identifier and use it anywhere in our code, just like a normal function.

In [3]:
zero = lambda x: 0
print zero(10000)

0


However, we can make more interesting lambda expression but actually manipulating the input. For example we can multiple the input by 2

In [4]:
double = lambda x: x*2
print double(2)
print double(4)

4
8


Lambda expressions in python support multiple arguments, such as

In [5]:
add = lambda x, y: x + y
add(5,7)

12

However, as we shall see, lambda calculus does not support this so we will implement a technique known as [currying](https://en.wikipedia.org/wiki/Currying). The basic concept of currying is converting a multiple-argument function into a sequence of single argument functions. This can be done easily in python as follows,

In [6]:
add = lambda x: lambda y: x + y
add(5)(7)

12

Note: the syntax of how we call the function changes. We must call the function once per argument. In fact, our `lambda x` function is a function that returns a function. To see this we can call the function with only one argument.

In [7]:
add_5 = add(5)
print add_5
print add_5(7)

<function <lambda> at 0x7ff00044fcf8>
12


Functions that return other functions are known as [higher-order functions, functionals, or functors](https://en.wikipedia.org/wiki/Higher-order_function). This is a central idea in the lambda calculus and will be used extensively throughout this document.

## Lambda Calculus

Formally, the lambda calculus can be defined as follows

* a set of variables $x, y, z$ ...
* an abstraction symbol $\lambda$ together with .
* parenthesis ()

The lambda calculus is then defined inductively using these building-blocks. Let $\wedge$ represent all valid lambda expressions.

1. if $x$ is a variable then $x \in \wedge$
2. if $x$ is a variable and $M \in \wedge$ then $(\lambda x. M) \in \wedge$ (abstraction operator)
3. if $M, N \in \wedge$ then $(M N) \in \wedge$ (application operator)

This is all that is required for lambda calculus! You're probably saying at this point, "that this definition is hopelessly abstract there is no way that I could do anything with that language." So, let's start out with some of the simplest elements of the lambda calculus. Doing so, we will also follow standard conventions for lambda calculus which are

* Outermost parentheses are dropped: $M N$ instead of $(M N)$
* Applications are assumed to be left associative: $M N P$ may be written instead of $((M N) P)$
* The body of an abstraction extends as far right as possible: $\lambda x.M N$ means $\lambda x.(M N)$ and not $(\lambda x.M) N$

Since, $x \in \wedge$ by axiom 1, we have that $(\lambda x. x) \in \wedge$ by axiom 2. This is the simplest lambda function you can write in the untyped lambda calculus. The function is typically called the identity and is denoted by
$$ id = \lambda x.x $$
The python equivalent is

In [8]:
Id = lambda x: x

Suppose $y \in \wedge$ is another variable, then we can apply axiom 3 to see that
$$ id\ y = \lambda x.x\ y = y$$
In python,

In [9]:
y = "some lambda expression"
Id(y)

'some lambda expression'

Now, you might say "hey you cheated! you started off with `y` a string, and that wasn't a lambda expression". You are correct, so now let's start from the very beginning.

## Booleans and Logic

Since we want to avoid using types like python's `True` and `False`, we must make our own functional notions of what it means to be true or false. We will define `True` and `False` as follows

$$ True = \lambda x. \lambda y. x$$
$$ False = \lambda x. \lambda y. y$$

Now those expresions make look totally contrived, but if we think about it from the perspective of control flow and the `if else` statement, we realize that it aligns very well with how we would like `if else` to behave. $True$ takes in two options and choses the first, just like an `if else` evaluating the `if` block when a statement is true. $False$ on the other hand takes in two options and choses the second, just like an `if else` evaluating the `else` block when a statement is false. In python we have

In [10]:
true = lambda x: lambda y: x
false = lambda x: lambda y: y

We will also create a little helper python function to make readable output for booleans. But this is strictly for python and not part of the lambda calculus.

In [11]:
to_py_bool = lambda x: True if x == true else False
print to_py_bool(true)
print to_py_bool(false)

True
False


Now we would like to implement some type of logic. Let's first start with "and".

$$ And = \lambda p. \lambda q. p q p $$

We observe that if $p$ is $True$ then we will go ahead and test $q$,

$$ True\ q\ True = \lambda x. \lambda y.x\ (q) (True) = q $$

In this case we would choose either $True$ or $False$ based on q.

Similarly, if $p$ if $False$ then we automatically choose $p$ which is $False$ 

$$ False\ q\ False = \lambda x. \lambda y.y\ (q)(False) = False $$

If you're not convinced, we can implement it in python and check the truth table.

In [12]:
And = lambda p: lambda q: p(q)(p)

print to_py_bool(And(true)(true))
print to_py_bool(And(true)(false))
print to_py_bool(And(false)(true))
print to_py_bool(And(false)(false))

True
False
False
False


Now let us definte "Or"

$$ Or = \lambda p. \lambda q. p p q $$

We observe that if $p$ is $True$ then we will go ahead and chose $p$,

$$ True\ True\ q = \lambda x. \lambda y.x\ (True) (q) = True $$

Similarly, if $p$ if $False$ then we go ahead and test $q$

$$ False\ False\ q = \lambda x. \lambda y.y\ (False)(q) = q $$

Let's implement it in python and check the truth table.

In [13]:
Or = lambda p: lambda q: p(p)(q)

print to_py_bool(Or(true)(true))
print to_py_bool(Or(true)(false))
print to_py_bool(Or(false)(true))
print to_py_bool(Or(false)(false))

True
True
True
False


To complete the basic boolean operators, we need a Not operator.

$$ Not = \lambda p. p False True $$

This is fairly straight-forward. If p is $True$ you will choose $False$

$$ True\ False\ True = \lambda x. \lambda y. x\ (False)(True) = False $$

and If p is $False$ you will choose $True$.

$$ False\ False\ True = \lambda x. \lambda y. y\ (False)(True) = True $$

Let's implement it in python

In [14]:
Not = lambda p: p(false)(true)

print to_py_bool(Not(true))
print to_py_bool(Not(false))

False
True


Now we can create more complicated expressions like "Nand" and "Xor"

$$ Nand = \lambda p.\lambda q.Not(And(p)(q)) $$

$$ Xor = \lambda p.\lambda q.p(Not(q))(q)$$

Let's check the python

In [15]:
Nand = lambda p: lambda q: Not(And(p)(q))

print to_py_bool(Nand(true)(true))
print to_py_bool(Nand(true)(false))
print to_py_bool(Nand(false)(true))
print to_py_bool(Nand(false)(false))

False
True
True
True


In [16]:
Xor = lambda p: lambda q: p(Not(q))(q)

print to_py_bool(Xor(true)(true))
print to_py_bool(Xor(true)(false))
print to_py_bool(Xor(false)(true))
print to_py_bool(Xor(false)(false))

False
True
True
False


Now we have all the tools to create arbitrarily complex boolean expressions in the lambda calculus. We simply do this by making new lambda functions combining the functions that we currently have.